In [5]:
import pandas as pd

filteredDf = pd.read_csv('attribute_list_filtered.csv')
filteredDf.head()

,file_name,hair_color,eyeglasses,smiling,young,human
0,1,1,-1,1,1,-1
1,2,4,-1,1,1,1
2,3,5,-1,1,-1,-1
3,7,2,-1,1,1,-1
4,8,3,-1,1,1,-1


In [8]:
from sklearn.preprocessing import OneHotEncoder

#one-hot-enconding:
def get_one_hot():
    df = pd.read_csv('attribute_list_filtered.csv')
    fileNameDf = df['file_name']
    df = df.drop('file_name', 1)
    enc = OneHotEncoder(categories='auto')
    enc.fit(df)
    onehotlabels = enc.transform(df).toarray()
    print(enc.get_feature_names())
    print(onehotlabels.shape)
    oneHotCols = ["hair_-1", "hair_0", "hair_1", "hair_2", "hair_3", "hair_4",
              "hair_5", "eyeglasses_-1", "eyeglasses_1", "smiling_-1", "smiling_1", 
              "young_-1", "young_1", "human_-1", "human_1"]
    oneHotDf = pd.DataFrame(onehotlabels, columns = oneHotCols)
    oneHotDf = pd.concat([fileNameDf, oneHotDf], axis=1)
    oneHotDf.to_csv('attribute_list_filtered_onehot.csv', index=False)
    return oneHotDf

oneHotDf = get_one_hot()

['x0_-1' 'x0_0' 'x0_1' 'x0_2' 'x0_3' 'x0_4' 'x0_5' 'x1_-1' 'x1_1' 'x2_-1'
 'x2_1' 'x3_-1' 'x3_1' 'x4_-1' 'x4_1']
(4565, 15)


In [30]:
len(oneHotDf)

4565

In [15]:
#data augmentation to prevent over-fitting example:
'''https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html'''
import os
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

img = load_img('./images/2.png')
img_arr = img_to_array(img) #shape: (3, 256, 256)
img_arr = img_arr.reshape((1,)+img_arr.shape) #adds extra dimension, shape: (1,3,256,256)

#generate batches of randomly transformed images, save in transformed dir
if os.path.isdir('transformed')==False:
    os.mkdir('transformed')

i = 0
flow = datagen.flow(img_arr, batch_size=1,
                    save_to_dir='transformed', 
                    save_prefix='2', save_format='png')
for batch in flow:
    i+=1
    if i>20:
        break

In [26]:
#splitting into train, validation, and test sets:
import numpy as np
def split_dataset_random(df, validation_split=0.2, test_split=0.2, seed=0):
    np.random.seed(seed) #if the same seed is used the same random split will be produced
    train_split = 1 - validation_split - test_split
    indeces_array = [int(train_split*len(df)), int((1-test_split)*len(df))]
    '''example, [2, 3] would, for axis=0, result in

        ary[:2]
        ary[2:3]
        ary[3:]

    '''
    train, validate, test = np.split(df.sample(frac=1), indeces_array)
    return train, validate, test

train, validate, test = split_dataset_random(oneHotDf)
print("train length: ", len(train), "| ratio: ", len(train)/len(oneHotDf))
print("validate length: ", len(validate), "| ratio: ", len(validate)/len(oneHotDf))
print("test length: ", len(test), "| ratio: ", len(test)/len(oneHotDf))

train length:  2739 | ratio:  0.6
validate length:  913 | ratio:  0.2
test length:  913 | ratio:  0.2


,file_name,hair_-1,hair_0,hair_1,hair_2,hair_3,hair_4,hair_5,eyeglasses_-1,eyeglasses_1,smiling_-1,smiling_1,young_-1,young_1,human_-1,human_1
3392,3711,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0


In [34]:
#Move images to train, test, validate directories according to split:
import shutil
import os

def move_split_images(train, validate, test):
    imagesPath = './images/'
    trainPath = './images/train/'
    validatePath = './images/validate/'
    testPath = './images/test/'
    newPaths = [trainPath, validatePath, testPath]
    dfList = [train, validate, test]
    for path in newPaths:
        if os.path.isdir(path)==False:
            os.mkdir(path)
    
    for df in dfList:
        for idx, row in df.iterrows():
            fileName = str(int(row['file_name']))+'.png'
            filePath = os.path.join(imagesPath, fileName) 
            if(df.equals(test)):
                newFilePath = os.path.join(testPath, fileName)
            if(df.equals(train)):
                newFilePath = os.path.join(trainPath, fileName)
            if(df.equals(validate)):
                newFilePath = os.path.join(validatePath, fileName)
            if os.path.isfile(filePath):
                shutil.move(filePath, newFilePath)
            else:
                print(filePath)

move_split_images(train, validate, test)

./images/3711.png
./images/1470.png
./images/4370.png
./images/4173.png
./images/3345.png
./images/4927.png
./images/3565.png
./images/2866.png
./images/3274.png
./images/2799.png
./images/601.png
./images/339.png
./images/4519.png
./images/976.png
./images/587.png
./images/2530.png
./images/3363.png
./images/4527.png
./images/2682.png
./images/1547.png
./images/4097.png
./images/1726.png
./images/4207.png
./images/3353.png
./images/142.png
./images/3722.png
./images/2868.png
./images/423.png
./images/4687.png
./images/2448.png
./images/4469.png
./images/4092.png
./images/320.png
./images/4118.png
./images/3562.png
./images/554.png
./images/646.png
./images/4005.png
./images/740.png
./images/3077.png
./images/3797.png
./images/4813.png
./images/235.png
./images/598.png
./images/2446.png
./images/2550.png
./images/988.png
./images/1083.png
./images/2143.png
./images/874.png
./images/335.png
./images/4171.png
./images/4303.png
./images/733.png
./images/244.png
./images/3377.png
./images/

./images/4555.png
./images/2099.png
./images/2798.png
./images/4852.png
./images/3850.png
./images/4530.png
./images/3609.png
./images/612.png
./images/1734.png
./images/3607.png
./images/1283.png
./images/2852.png
./images/1843.png
./images/381.png
./images/1498.png
./images/4238.png
./images/1411.png
./images/984.png
./images/314.png
./images/515.png
./images/3342.png
./images/1690.png
./images/2248.png
./images/2416.png
./images/1934.png
./images/3109.png
./images/1884.png
./images/3436.png
./images/4454.png
./images/2650.png
./images/147.png
./images/902.png
./images/643.png
./images/1330.png
./images/1677.png
./images/1082.png
./images/1720.png
./images/4468.png
./images/2880.png
./images/3434.png
./images/1830.png
./images/1637.png
./images/2408.png
./images/2493.png
./images/3821.png
./images/2541.png
./images/2529.png
./images/4693.png
./images/2212.png
./images/1472.png
./images/3224.png
./images/2253.png
./images/453.png
./images/3030.png
./images/2944.png
./images/3140.png
.

./images/4162.png
./images/4779.png
./images/1125.png
./images/192.png
./images/59.png
./images/1058.png
./images/74.png
./images/4565.png
./images/4553.png
./images/2495.png
./images/1317.png
./images/442.png
./images/2645.png
./images/3338.png
./images/3613.png
./images/1958.png
./images/2017.png
./images/2375.png
./images/1855.png
./images/509.png
./images/3918.png
./images/2961.png
./images/1701.png
./images/2324.png
./images/1890.png
./images/4073.png
./images/997.png
./images/2721.png
./images/426.png
./images/3310.png
./images/1725.png
./images/4947.png
./images/2812.png
./images/4459.png
./images/2871.png
./images/209.png
./images/1644.png
./images/218.png
./images/3920.png
./images/4931.png
./images/4096.png
./images/3617.png
./images/624.png
./images/1528.png
./images/4500.png
./images/4421.png
./images/4937.png
./images/2566.png
./images/4561.png
./images/3329.png
./images/1027.png
./images/4972.png
./images/4761.png
./images/2522.png
./images/1374.png
./images/4837.png
./im

./images/3344.png
./images/1037.png
./images/1673.png
./images/1309.png
./images/3157.png
./images/3347.png
./images/3000.png
./images/1515.png
./images/4147.png
./images/4455.png
./images/3164.png
./images/789.png
./images/4274.png
./images/2610.png
./images/2247.png
./images/3429.png
./images/3913.png
./images/2139.png
./images/623.png
./images/3886.png
./images/828.png
./images/4810.png
./images/725.png
./images/2720.png
./images/355.png
./images/4486.png
./images/2769.png
./images/1260.png
./images/2631.png
./images/472.png
./images/2354.png
./images/4187.png
./images/2141.png
./images/4552.png
./images/4825.png
./images/3646.png
./images/664.png
./images/1975.png
./images/2368.png
./images/2717.png
./images/4362.png
./images/1151.png
./images/869.png
./images/1406.png
./images/4799.png
./images/3539.png
./images/4887.png
./images/133.png
./images/861.png
./images/1087.png
./images/4672.png
./images/2592.png
./images/827.png
./images/4052.png
./images/1938.png
./images/3413.png
./i

./images/2675.png
./images/4564.png
./images/87.png
./images/1565.png
./images/287.png
./images/216.png
./images/2511.png
./images/799.png
./images/4366.png
./images/3932.png
./images/4072.png
./images/3496.png
./images/4195.png
./images/478.png
./images/2663.png
./images/1050.png
./images/3046.png
./images/1781.png
./images/4739.png


In [16]:
#testing cnn
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(3, 150, 150)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

ValueError: Negative dimension size caused by subtracting 2 from 1 for 'max_pooling2d_1/MaxPool' (op: 'MaxPool') with input shapes: [?,1,148,32].